In [3]:
import numpy as np
import pandas as pd
import pickle
import tsplib95

In [4]:
def seed():
    np.random.seed(42)

In [5]:
def load_path(file_path):
    '''Load the optimal route.'''
    dic = tsplib95.load(file_path)
    return dic.tours[0]

optimal_route = load_path("TSP-Configurations\eil51.opt.tour.txt")
print(optimal_route)

def load_file(file_path): 
    '''Load the coordinates of each dimension.'''
    dic = tsplib95.load(file_path)

    x_coordinates = [coords[0] for _, coords in list(dic.node_coords.items())]
    y_coordinates = [coords[1] for _, coords in list(dic.node_coords.items())]
    
    return np.array(x_coordinates), np.array(y_coordinates)

x_coordinates, y_coordinates = load_file("TSP-Configurations\eil51.tsp.txt")
print(x_coordinates)
print(y_coordinates)

[1, 22, 8, 26, 31, 28, 3, 36, 35, 20, 2, 29, 21, 16, 50, 34, 30, 9, 49, 10, 39, 33, 45, 15, 44, 42, 40, 19, 41, 13, 25, 14, 24, 43, 7, 23, 48, 6, 27, 51, 46, 12, 47, 18, 4, 17, 37, 5, 38, 11, 32]
[37 49 52 20 40 21 17 31 52 51 42 31  5 12 36 52 27 17 13 57 62 42 16  8
  7 27 30 43 58 58 37 38 46 61 62 63 32 45 59  5 10 21  5 30 39 32 25 25
 48 56 30]
[52 49 64 26 30 47 63 62 33 21 41 32 25 42 16 41 23 33 13 58 42 57 57 52
 38 68 48 67 48 27 69 46 10 33 63 69 22 35 15  6 17 10 64 15 10 39 32 55
 28 37 40]


In [6]:
def get_distance(route, x_coordinates, y_coordinates): 
    '''Calculate the total distance with given route.'''
    distance = 0
    for idx, i in enumerate(route[1:]): 
        x_distance = np.abs(x_coordinates[i-1] - x_coordinates[route[idx]-1])
        y_distance = np.abs(y_coordinates[i-1] - y_coordinates[route[idx]-1])
        distance += np.sqrt(x_distance**2 + y_distance**2)
    x_distance =  np.abs(x_coordinates[route[-1]-1] - x_coordinates[route[0]-1])
    y_distance =  np.abs(y_coordinates[route[-1]-1] - y_coordinates[route[0]-1])
    distance += np.sqrt(x_distance**2 + y_distance**2)
    
    return distance

distance = get_distance(optimal_route, x_coordinates, y_coordinates)
print(optimal_route)
print(distance)


[1, 22, 8, 26, 31, 28, 3, 36, 35, 20, 2, 29, 21, 16, 50, 34, 30, 9, 49, 10, 39, 33, 45, 15, 44, 42, 40, 19, 41, 13, 25, 14, 24, 43, 7, 23, 48, 6, 27, 51, 46, 12, 47, 18, 4, 17, 37, 5, 38, 11, 32]
429.98331198338406


In [7]:
def get_distance_matrix(x_coordinates, y_coordinates): 
    dimension = len(x_coordinates)
    distance_matrix = np.zeros((dimension, dimension))
    for i in range(dimension): 
        distance_matrix[i] = (x_coordinates - x_coordinates[i])**2
        distance_matrix[i] += (y_coordinates - y_coordinates[i])**2
    
    return np.sqrt(distance_matrix)

distance_matrix = get_distance_matrix(x_coordinates, y_coordinates)

In [8]:
optimal_route_ = (optimal_route - np.ones(51)).astype(int)
reshape_route = np.array((optimal_route_[:-1], optimal_route_[1:])).T
reshape_route = np.vstack((reshape_route, np.array([[optimal_route_[-1], optimal_route_[0]]])))
distance_matrix[reshape_route[:, 0], reshape_route[:, 1]].sum()

429.983311983384

In [9]:
rng = np.random.default_rng()
random_path = rng.choice(np.arange(1, 52), size=51, replace=False)

distance = get_distance(random_path, x_coordinates, y_coordinates)
print(distance)

1580.705088554159


In [101]:
class SA(): 
    def __init__(self, dimension, num_i, temperature=1, MC_length=1):
        if dimension == 51: 
            self.dimension = 51
            self.x_coordinates, self.y_coordinates = load_file("TSP-Configurations\eil51.tsp.txt")
        elif dimension == 280:
            self.dimension = 280
            self.x_coordinates, self.y_coordinates = load_file("TSP-Configurations\a280.tsp.txt")
        elif dimension == 442: 
            self.dimension = 442
            self.x_coordinates, self.y_coordinates = load_file("TSP-Configurations\pcb442.tsp.txt")
        else: 
            raise ValueError('No such file')
        
        rng = np.random.default_rng()
        self.route = rng.choice(range(self.dimension), size=self.dimension, replace=False)
        self.distance_matrix = self.get_distance_matrix()
        self.distance = self.get_distance(self.route)
        self.T = temperature
        self.coef_cooling_lin = temperature * (1 - 0.9 ** num_i) / num_i
        self.coef_cooling_log = np.log(2)
        self.num_i = num_i
        self.MC_length = MC_length

    def get_distance_matrix(self): 
        distance_matrix = np.zeros((self.dimension, self.dimension))
        for i in range(self.dimension): 
            distance_matrix[i] = (self.x_coordinates - self.x_coordinates[i])**2
            distance_matrix[i] += (self.y_coordinates - self.y_coordinates[i])**2
        
        return np.sqrt(distance_matrix)
    
    def get_distance(self, route):
        reshape_route = np.array((route[:-1], route[1:])).T
        reshape_route = np.vstack((reshape_route, np.array([[route[-1], route[0]]])))

        return self.distance_matrix[reshape_route[:, 0], reshape_route[:, 1]].sum()
    
    def two_opt(self): 
        position_1, position_2 = np.sort(rng.choice(range(self.dimension), size=2, replace=False))
        new_route = np.copy(self.route)
        new_route[position_1:position_2+1] = self.route[np.arange(position_2, position_1-1, -1)]
        new_distance = self.get_distance(new_route) 

        return new_route, new_distance

    def acceptance_criteria_sa(self, new_route, new_distance): 
        '''acceptance criteria for simulated annealing algorithm'''
        random = np.random.rand()
        if random <= min(np.exp(-(new_distance-self.distance)/self.T),1): 
            self.route = new_route
            self.distance = new_distance

    def acceptance_criteria_hc(self, new_route, new_distance): 
        '''acceptance criteria for hill climbing algorithm'''
        if new_distance < self.distance:
            self.route = new_route
            self.distance = new_distance

    def cooling_schedule_lin(self): 
        # Balance the temperature after num_i iterations of linear and geometric cooling schedules
        self.T -= self.coef_cooling_lin

    def cooling_schedule_geo(self): 
        self.T *= 0.9

    def cooling_schedule_log(self): 
        self.current_iteration += 1
        self.T = self.coef_cooling_log / np.log(1+self.current_iteration)

    def run_simulation_sa_lin(self): 
        distance_list = np.zeros(self.num_i)
        for i in range(self.num_i): 
            new_route, new_distance = self.two_opt()
            self.acceptance_criteria_sa(new_route, new_distance)
            self.cooling_schedule_lin()
            distance_list[i] = self.distance
        
        return distance_list, self.route
    
    def run_simulation_sa_geo(self): 
        distance_list = np.zeros(self.num_i)
        counter = 0
        for i in range(self.num_i): 
            new_route, new_distance = self.two_opt()
            self.acceptance_criteria_sa(new_route, new_distance)
            if counter == self.MC_length: 
                self.cooling_schedule_geo()
                counter = 0
            distance_list[i] = self.distance
            counter += 1
        
        return distance_list, self.route
    
    def run_simulation_sa_log(self): 
        distance_list = np.zeros(self.num_i)
        counter = 0
        self.current_iteration = 0
        for i in range(self.num_i): 
            new_route, new_distance = self.two_opt()
            self.acceptance_criteria_sa(new_route, new_distance)
            if counter == self.MC_length: 
                self.cooling_schedule_log()
                counter = 0
            distance_list[i] = self.distance
        
        return distance_list, self.route
    
    def run_simulation_hc(self): 
        distance_list = np.zeros(self.num_i)
        for i in range(self.num_i): 
            new_route, new_distance = self.two_opt()
            self.acceptance_criteria_hc(new_route, new_distance)
            distance_list[i] = self.distance
        
        return distance_list, self.route
    

In [102]:
def run_multiple_simulation(dimension=51, num_i=1000, temperature=1, num_run=50, MC_length=1, alg_type='SA', SA_type='geo', save_file=None, load_file=None): 
    seed()
    results = np.zeros((num_run, num_i))
    if alg_type == 'SA': 

        file_prefix = f'dimension_{dimension}_solved_by_SA_with_type_{SA_type}_initial_temperature_{temperature}_MClength_{MC_length}_num_i_{num_i}_num_run_{num_run}'
        if load_file: 
            with open(f'data/{file_prefix}_results.pkl', 'rb') as f:
                results = pickle.load(f)
            with open(f'data/{file_prefix}_route.pkl', 'rb') as f:
                route = pickle.load(f)
            return results, route

        if SA_type == 'lin': 
            for i in range(num_run): 
                    results[i,:], route = SA(dimension, num_i, temperature, MC_length).run_simulation_sa_lin()
        elif SA_type == 'geo': 
            for i in range(num_run): 
                results[i,:], route = SA(dimension, num_i, temperature, MC_length).run_simulation_sa_geo()
        elif SA_type == 'log': 
            for i in range(num_run): 
                    results[i,:], route = SA(dimension, num_i, temperature, MC_length).run_simulation_sa_log()
    
    elif alg_type == 'HC':
        file_prefix = f'dimension_{dimension}_solved_by_HC_with_num_i_{num_i}_num_run_{num_run}'
        if load_file: 
            with open(f'data/{file_prefix}_results.pkl', 'rb') as f:
                results = pickle.load(f)
            with open(f'data/{file_prefix}_route.pkl', 'rb') as f:
                route = pickle.load(f)

        for i in range(num_run): 
                results[i,:], route = SA(dimension, num_i).run_simulation_hc()
    
    if save_file: 
        with open(f'data/{file_prefix}_results.pkl', 'wb') as f: 
             pickle.dump(results, f)
        with open(f'data/{file_prefix}_route.pkl', 'wb') as f: 
             pickle.dump(route, f)
    
    return results, route
        
# results, route = run_multiple_simulation(dimension=51, temperature=1, num_i=100000, num_run=50, save_file='True')

In [64]:
results[:,-1]

array([457.64775113, 466.19045167, 466.43087728, 472.8874379 ,
       450.52710244, 463.95196249, 448.98105613, 471.38354623,
       457.57893267, 440.37426993, 458.01125323, 461.73323995,
       453.98461202, 459.98020184, 456.80882056, 457.97862347,
       468.37413352, 482.23016586, 452.70476688, 466.82086253,
       450.41788744, 452.68170282, 453.70209401, 457.9055757 ,
       452.69555633, 445.98518203, 468.75002464, 454.53799378,
       457.69968874, 470.28149949, 442.81920154, 451.68329914,
       460.68076753, 457.3743302 , 466.71317654, 459.56726329,
       470.79738307, 465.49705893, 439.49210959, 463.35691607,
       463.02287232, 464.10207534, 461.96539403, 455.28769075,
       451.56714794, 459.98194698, 455.75170902, 451.3202248 ,
       465.28987683, 448.12098933])

In [98]:
results_test, route = run_multiple_simulation(dimension=51, num_i=100000, temperature=1, num_run=50, load_file=True)
print(np.mean(results_test[:,-1]))

458.67257411909543


In [103]:
results_hc, route = run_multiple_simulation(dimension=51, num_i=100000, alg_type='HC', save_file=True)
print(np.mean(results_hc[:,-1]))

460.71368218913165


In [106]:
print(route)

[25  7 30 27  2 35 34 19 28 20 49 33 29  9 38 32 44 14 43 36 16 11 46 17
  3 41 18 39 40 12 24 13  5 26 50 45 31 10 37  4 48  8 15  1 21  0 47 22
 23 42  6]


In [66]:
results, route = run_multiple_simulation(dimension=51, temperature=1, num_i=200000, num_run=50, save_file='True')

C:\Users\lishu\AppData\Local\Temp\ipykernel_20656\341573099.py:50: RuntimeWarning: overflow encountered in exp
  if random <= min(np.exp(-(new_distance-self.distance)/self.T),1):
C:\Users\lishu\AppData\Local\Temp\ipykernel_20656\341573099.py:50: RuntimeWarning: overflow encountered in scalar divide
  if random <= min(np.exp(-(new_distance-self.distance)/self.T),1):


In [57]:
import numpy as np
import pickle
from random import seed
from concurrent.futures import ProcessPoolExecutor, as_completed

def run_single_simulation(dimension, num_i, temperature, MC_length, SA_type):
    """Run a single simulation based on the specified SA type."""
    if SA_type == 'lin':
        return SA(dimension, num_i, temperature, MC_length).run_simulation_sa_lin()
    elif SA_type == 'geo':
        return SA(dimension, num_i, temperature, MC_length).run_simulation_sa_geo()
    elif SA_type == 'log':
        return SA(dimension, num_i, temperature, MC_length).run_simulation_sa_log()
    else:
        raise ValueError("Invalid SA_type provided.")

def run_multiple_simulation(dimension=51, temperature=1, num_i=1000, num_run=50, MC_length=1, alg_type='SA', SA_type='geo', save_file=None, load_file=None): 
    seed()
    results = np.zeros((num_run, num_i))
    routes = []

    file_prefix = f'dimension_{dimension}_solved_by_{alg_type}_with_type_{SA_type}_initial_temperature_{temperature}_MClength_{MC_length}_num_i_{num_i}_num_run_{num_run}'
    
    if load_file: 
        with open(f'data/{file_prefix}_results.pkl', 'rb') as f:
            results = pickle.load(f)
        with open(f'data/{file_prefix}_route.pkl', 'rb') as f:
            routes = pickle.load(f)
        return results, routes

    # Use ProcessPoolExecutor for parallel execution
    with ProcessPoolExecutor() as executor:
        futures = []
        
        for i in range(num_run):
            futures.append(executor.submit(run_single_simulation, dimension, num_i, temperature, MC_length, SA_type))

        for i, future in enumerate(as_completed(futures)):
            try:
                results[i], route = future.result()
                routes.append(route)
            except Exception as e:
                print(f"Simulation {i} failed with error: {e}")
                routes.append(None)  # Append None or handle it as needed

    if save_file: 
        with open(f'data/{file_prefix}_results.pkl', 'wb') as f: 
            pickle.dump(results, f)
        with open(f'data/{file_prefix}_route.pkl', 'wb') as f: 
            pickle.dump(routes, f)
    
    return results, routes

# Example usage
results, routes = run_multiple_simulation(dimension=51, temperature=1, num_i=1000, num_run=50)


Simulation 0 failed with error: A process in the process pool was terminated abruptly while the future was running or pending.
Simulation 1 failed with error: A process in the process pool was terminated abruptly while the future was running or pending.
Simulation 2 failed with error: A process in the process pool was terminated abruptly while the future was running or pending.
Simulation 3 failed with error: A process in the process pool was terminated abruptly while the future was running or pending.
Simulation 4 failed with error: A process in the process pool was terminated abruptly while the future was running or pending.
Simulation 5 failed with error: A process in the process pool was terminated abruptly while the future was running or pending.
Simulation 6 failed with error: A process in the process pool was terminated abruptly while the future was running or pending.
Simulation 7 failed with error: A process in the process pool was terminated abruptly while the future was runn

In [47]:
results[:,-1]

array([637.75211539, 578.69151819, 720.73535242, 638.7405202 ,
       624.28580211, 589.92496463, 572.71973601, 598.99428372,
       599.98212449, 594.51428711, 566.73844944, 622.94010911,
       588.07125577, 579.51708115, 613.32011183, 657.58486839,
       549.41560254, 596.92286225, 603.96503282, 655.61741506,
       559.27111828, 595.0592686 , 560.98325587, 612.11226599,
       585.33472963, 588.12076386, 597.18882165, 594.06331851,
       542.37322116, 569.7934238 , 585.94947629, 606.98811117,
       563.86994194, 657.15074856, 594.33348446, 576.9998818 ,
       579.11019881, 595.90437764, 601.19005283, 610.54474086,
       548.55301971, 585.49187794, 603.13063592, 561.30348857,
       639.5315207 , 578.63066769, 591.76157366, 567.89563256,
       564.96736096, 605.7846144 ])

In [48]:
results_1, route = run_multiple_simulation(dimension=51, temperature=1, num_i=1000, num_run=50)
results_1[:,-1]

C:\Users\lishu\AppData\Local\Temp\ipykernel_20656\341573099.py:50: RuntimeWarning: overflow encountered in exp
  if random <= min(np.exp(-(new_distance-self.distance)/self.T),1):


array([589.15518776, 627.8609859 , 564.70639199, 640.023059  ,
       605.89905068, 618.21933932, 573.83262779, 558.03408979,
       605.18901261, 573.14709466, 584.90897637, 643.80899949,
       608.74743534, 633.91419691, 567.92928587, 621.3405967 ,
       618.43307202, 582.55898874, 559.46160968, 601.91535477,
       604.00904615, 613.08335125, 643.66252278, 571.13301806,
       599.27909235, 597.79464743, 556.22541009, 549.26545033,
       569.93504265, 597.46846947, 586.69986448, 620.64492741,
       537.13907316, 574.13315307, 649.20900553, 540.01189231,
       618.72985605, 576.1525695 , 641.45539181, 623.14326591,
       581.83794012, 667.88512501, 623.1244634 , 640.89159965,
       573.8263888 , 556.51332509, 589.41050425, 601.79708006,
       636.95574487, 609.30881993])

In [ ]:
results_30, route = run_multiple_simulation(dimension=51, temperature=1, num_i=1000, num_run=50, MC_length=30)
results_30[:,-1]

array([[1693.70665785, 1693.70665785, 1687.4097226 , ...,  614.42111273,
         614.42111273,  614.42111273],
       [1638.43375338, 1638.43375338, 1613.73732301, ...,  639.52127356,
         639.52127356,  639.52127356],
       [1567.59545725, 1567.59545725, 1567.59545725, ...,  564.57002477,
         564.57002477,  564.57002477],
       ...,
       [1622.68071182, 1593.29012011, 1560.11129625, ...,  637.09179133,
         637.09179133,  637.09179133],
       [1733.88655962, 1696.88596313, 1696.88596313, ...,  672.13366455,
         672.13366455,  672.13366455],
       [1768.33807131, 1737.07657474, 1737.07657474, ...,  579.68732554,
         579.68732554,  579.68732554]])